In [17]:
import os
import numpy as np
import IPython
import copy
from shutil import copyfile

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import base


import sys
sys.dont_write_bytecode=True

PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

from influence.inceptionModel import BinaryInceptionModel
from influence.logisticRegressionWithLBFGS import LogisticRegressionWithLBFGS
import influence.experiments
from influence.dataset import DataSet
# from influence.dataset_poisoning import iterative_attack, select_examples_to_attack, get_projection_to_box_around_orig_point, generate_inception_features
from influence.iter_attack import iterative_attack, select_examples_to_attack, get_projection_to_box_around_orig_point, generate_inception_features
from influence.Progress import *

from load_animals import *

from skimage import io

%load_ext autoreload
%autoreload 2

from data_poisoning import data_poisoning

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
data_selected = ['dog', 'fish']
num_train_ex_per_class, num_test_ex_per_class = 900, 300
use_IF = True
target_test_idx = [45]
num_to_perterb = 1

In [19]:
# Find the cleaned dataset
data_sets = load_animals(
                    num_train_ex_per_class=num_train_ex_per_class, 
                    num_test_ex_per_class=num_test_ex_per_class,
                    classes=data_selected)

img_side = 299
num_channels = 3
 
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [1000, 10000]

weight_decay = 0.001

num_classes = len(data_selected)
max_lbfgs_iter = 1000
batch_size = 50

dataset_name = 'transferbility_%s_%s' % (num_train_ex_per_class, num_test_ex_per_class)

Loading animals from disk...
../data/dataset_dog-fish_train-900_test-300.npz


In [20]:
# InceptionResNet
full_graph = tf.Graph()
with full_graph.as_default():
    full_model_name = '%s_inceptionRes_wd-%s' % (dataset_name, weight_decay)
    full_model = BinaryInceptionModel(
        img_side=img_side,
        num_channels=num_channels,
        weight_decay=weight_decay,
        use_InceptionResNet=True,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=True,
        train_dir='output',
        log_dir='log',
        model_name=full_model_name)

self.logits Tensor("Shape:0", shape=(2,), dtype=int32)


[genericNeuralNet.py:191 -   get_vec_to_list_fn() ] Total number of parameters: 1536


wrong_labels_bool Tensor("Shape_2:0", shape=(2,), dtype=int32)
logits Tensor("Shape_3:0", shape=(2,), dtype=int32)
inception_features:  Tensor("flatten/Reshape:0", shape=(?, ?), dtype=float32)
x_poison_features:  Tensor("Gather:0", shape=(1, ?), dtype=float32)
t_target_features:  Tensor("Gather_1:0", shape=(1, ?), dtype=float32)
Lp:  Tensor("norm/Squeeze:0", shape=(), dtype=float32)
LP_gradient Tensor("strided_slice_1:0", shape=(268203,), dtype=float32)


In [21]:
full_model.retrain_and_get_weights(data_sets.train.x, data_sets.train.labels)

[genericNeuralNet.py:328 -     print_model_eval() ] Train loss (w reg) on all data: [0.01579112]
[genericNeuralNet.py:329 -     print_model_eval() ] Train loss (w/o reg) on all data: [0.00576179]
[genericNeuralNet.py:331 -     print_model_eval() ] Test loss (w/o reg) on all data: [0.02447189]
[genericNeuralNet.py:332 -     print_model_eval() ] Train acc on all data:  [1.]
[genericNeuralNet.py:333 -     print_model_eval() ] Test acc on all data:   [0.99166667]
[genericNeuralNet.py:335 -     print_model_eval() ] Norm of the mean of gradients: 2.501867e-06
[genericNeuralNet.py:336 -     print_model_eval() ] Norm of the params: 4.4786887


array([0.17748275, 0.06460591, 0.08047303, ..., 0.00543467, 0.05926711,
       0.03573376], dtype=float32)